#### Setting up Colab

To run this notebook on [Google's Colab](https://colab.research.google.com), you will need to perform the following steps.

Step 1. Install pyspark

Since pyspark isn't included in Colab's Python installation, you will need to install it each time you open this notebook.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=47525917b3b96a30b4f5fb73a81c625b2b7e1511e182df30d0b2af455ac3c32d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install composable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
  Attempting uninstall: toolz
    Found existing installation: toolz 0.12.0
    Uninstalling toolz-0.12.0:
      Successfully uninstalled toolz-0.12.0


Step 2. Download and unzip the data

Next, the easiest way to access the data from the module is to download and unzip.

In [3]:
!wget https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/data.zip

--2022-10-11 14:08:09--  https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/data.zip [following]
--2022-10-11 14:08:09--  https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14959102 (14M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  14.27M  --.-KB/s    in 0.1s    

2022-10-11 14:08:09 (120 MB/s) - ‘data.zip’ saved [14959102/14959102]



In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/Rochester_temps_2019.xlsx  
  inflating: __MACOSX/data/._Rochester_temps_2019.xlsx  
  inflating: data/uber-raw-data-jun14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-jun14-sample.csv  
  inflating: data/heroes_information.csv  
  inflating: __MACOSX/data/._heroes_information.csv  
  inflating: data/employee.csv       
  inflating: __MACOSX/data/._employee.csv  
  inflating: data/uber-raw-data-apr14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-apr14-sample.csv  
   creating: data/baseball/
  inflating: __MACOSX/data/._baseball  
  inflating: data/uber-raw-data-may14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-may14-sample.csv  
  inflating: data/auto_sales_apr.csv  
  inflating: __MACOSX/data/._auto_sales_apr.csv  
  inflating: data/uber-raw-data-sep14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-sep14-sample.csv  
  inflating: data/department.csv 

In [5]:
!wget https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/more_pyspark.py

--2022-10-11 14:08:10--  https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/more_pyspark.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/more_pyspark.py [following]
--2022-10-11 14:08:10--  https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/more_pyspark.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2786 (2.7K) [text/plain]
Saving to: ‘more_pyspark.py’

more_pyspark.py     100%[===================>]   2.72K  --.-KB/s    in 0s      

2022-10-11 14:08:10 (27.1 MB/s) - ‘more_pyspark.py’ saved [2786/2786]



# Concatenating Tables with Set-Like Operations in `pyspark`

Now let's look at combining tables with `union`, `intersect`, and `except` in `pyspark`.

## Example - Auto Sales in Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').config('spark.driver.host', 'localhost').getOrCreate()

22/10/27 22:44:24 WARN Utils: Your hostname, nn1448lr222 resolves to a loopback address: 127.0.1.1; using 172.17.25.238 instead (on interface eth0)
22/10/27 22:44:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/27 22:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/10/27 22:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [2]:
from more_pyspark import to_pandas
sales_apr = spark.read.csv("./data/auto_sales_apr.csv",  header=True, inferSchema=True)
sales_apr.collect() >> to_pandas

22/10/27 22:44:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_apr.csv


,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


In [3]:
sales_may = spark.read.csv("./data/auto_sales_may.csv",  header=True, inferSchema=True)
sales_may.collect() >> to_pandas

22/10/27 22:44:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_may.csv


,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


## `UNION ALL` in `pyspark`

Both `union` and `unionAll` area actually `UNION ALL`

In [4]:
(sales_apr
 .union(sales_may)
 .collect()
) >> to_pandas

22/10/27 22:45:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_apr.csv
22/10/27 22:45:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_may.csv


,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9
4,0,Ann,22,18,15,12
5,1,Bob,20,14,6,24
6,2,Yolanda,19,10,28,17
7,3,Xerxes,11,27,17,9


## `UNION/UNION DISTINCT` in `pyspark`

Use `distinct` to get the usual `UNION/UNION DISTINCT`

In [5]:
(sales_apr
 .union(sales_may)
 .distinct()
 .collect()
) >> to_pandas

22/10/27 22:45:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_may.csv
22/10/27 22:45:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_apr.csv


,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,2,Yolanda,19,8,32,15
1,3,Xerxes,12,23,18,9
2,1,Bob,19,12,17,20
3,0,Ann,22,18,15,12
4,1,Bob,20,14,6,24
5,2,Yolanda,19,10,28,17
6,3,Xerxes,11,27,17,9


## Adding a `month` column

As mentioned before, we really should add a month column here. Note that we need to use `lit` to add a *literal constant*

In [6]:
from pyspark.sql.functions import lit

(sales_apr.withColumn('month', lit('April'))
 .union(sales_may.withColumn('month', lit('May')))
 .collect()
) >> to_pandas

22/10/27 22:45:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_may.csv
22/10/27 22:45:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_apr.csv


,_c0,Salesperson,Compact,Sedan,SUV,Truck,month
0,0,Ann,22,18,15,12,April
1,1,Bob,19,12,17,20,April
2,2,Yolanda,19,8,32,15,April
3,3,Xerxes,12,23,18,9,April
4,0,Ann,22,18,15,12,May
5,1,Bob,20,14,6,24,May
6,2,Yolanda,19,10,28,17,May
7,3,Xerxes,11,27,17,9,May


## Performing `INTERSECT`

Note that `intersect` and `intersectAll` are synonymous.

In [7]:
sales_apr.intersect(sales_may).collect() >> to_pandas

22/10/27 22:46:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_may.csv
22/10/27 22:46:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Salesperson, Compact, Sedan, SUV, Truck
 Schema: _c0, Salesperson, Compact, Sedan, SUV, Truck
Expected: _c0 but found: 
CSV file: file:///home/wavessurfer/github-classroom/wsu-stat489/module-5-lectures-wavessurfer/data/auto_sales_apr.csv


,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12


## Performing a set difference with `exceptAll`

In [14]:
sales_apr.exceptAll(sales_may).collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,1,Bob,19,12,17,20
1,2,Yolanda,19,8,32,15
2,3,Xerxes,12,23,18,9


## <font color="red"> Exercise 2 </font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Read in `pyspark` dataframes for April and May.
2. Check that the information about the date/month is present, and add a column if necessary.
3. `union` method to combine these 2 data frames into one combined `pyspark df`

In [15]:
!ls ./data | grep uber

uber-raw-data-apr14-sample.csv
uber-raw-data-aug14-sample.csv
uber-raw-data-jul14-sample.csv
uber-raw-data-jun14-sample.csv
uber-raw-data-may14-sample.csv
uber-raw-data-sep14-sample.csv


In [10]:
# Your code here
uber_april = spark.read.csv("./data/uber-raw-data-apr14-sample.csv",  header=True, inferSchema=True)
uber_may = spark.read.csv("./data/uber-raw-data-may14-sample.csv",  header=True, inferSchema=True)

(uber_april.withColumn('month', lit('April'))
 .union(uber_may.withColumn('month', lit('May')))
 .collect()
) >> to_pandas

,Date/Time,Lat,Lon,Base,month
0,4/18/2014 21:38:00,40.7359,-73.9852,B02682,April
1,4/23/2014 15:19:00,40.7642,-73.9543,B02598,April
2,4/10/2014 7:15:00,40.7138,-74.0103,B02598,April
3,4/11/2014 15:23:00,40.7847,-73.9698,B02682,April
4,4/7/2014 17:26:00,40.6460,-73.7767,B02598,April
...,...,...,...,...,...
199995,5/23/2014 22:41:00,40.7341,-74.0084,B02682,May
199996,5/30/2014 6:53:00,40.6770,-74.0155,B02682,May
199997,5/3/2014 18:23:00,40.7710,-73.9636,B02682,May
199998,5/17/2014 13:16:00,40.7586,-73.9614,B02598,May


## <font color="red"> Exercise 3 -- Optional </font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a regular expression to create a `lambda` function that pulls the month from the files.
3. Read the 6 `pyspark` dataframes into a `dict` with keys equal to the month name and values containing the corresponding data frame.
4. Use a dictionary comprehension to add a month column to each `df`.
5. Use the accumulator pattern and the `union` method to combine these 6 data frames into one combined `pyspark df`
6. Inspect the head and compute the number of rows (use the `count` method)

In [ ]:
!ls ./data | grep uber

In [ ]:
# Your code here